# **1. 데이터 로드 및 전처리**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import re
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import os

In [6]:
# 토크나이저 및 모델 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")  # 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")  # 모델 로드

# 패딩 토큰 추가
special_tokens = {'pad_token': '[PAD]'}
tokenizer.add_special_tokens(special_tokens)  # 패딩 토큰 추가
model.resize_token_embeddings(len(tokenizer))  # 토크나이저 크기 재조정
model.config.pad_token_id = tokenizer.pad_token_id  # 패딩 토큰 ID 설정

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

## (1-1). 추가 학습 시

In [ ]:
# 모델 및 토크나이저 로드
model_path = "/content/drive/MyDrive/fine_tuned_kogpt2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# 패딩 토큰 추가
special_tokens = {'pad_token': '[PAD]'}
tokenizer.add_special_tokens(special_tokens)  # 패딩 토큰 추가
model.resize_token_embeddings(len(tokenizer))  # 토크나이저 크기 재조정
model.config.pad_token_id = tokenizer.pad_token_id  # 패딩 토큰 ID 설정

In [7]:
# 데이터 로드 및 전처리
df = pd.read_csv('/content/drive/MyDrive/EPL.csv')  # CSV 파일에서 데이터 로드
df = df.dropna().drop_duplicates()  # 결측치와 중복 행 제거

def clean_text(text):
    # 줄 바꿈 문자를 띄어쓰기로 대체
    text = re.sub(r'\n', ' ', text)
    # 영문자, 한글, 숫자, 공백, 괄호, 물음표를 제외한 모든 문자 제거
    text = re.sub(r'[^a-zA-Z가-힣0-9\s\(\)\?]', '', text)
    # 연속된 공백을 하나의 공백으로 대체
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    return text.strip()

df['Question'] = df['Question'].apply(clean_text)  # 질문 텍스트 정리
df['Answer'] = df['Answer'].apply(clean_text)  # 답변 텍스트 정리

# **2. 데이터셋 생성**

In [8]:
# 데이터셋 클래스 정의
class EPLDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.tokenizer = tokenizer  # 토크나이저 저장
        self.data = dataframe  # 데이터 저장
        self.max_length = max_length  # 최대 길이 설정

    def __len__(self):
        return len(self.data)  # 데이터셋의 길이 반환

    def __getitem__(self, index):
        question = self.data.iloc[index]['Question']  # 특정 인덱스의 질문 가져오기
        answer = self.data.iloc[index]['Answer']  # 특정 인덱스의 답변 가져오기

        # 질문과 답변을 결합
        full_text = f"질문: {question} 답변: {answer}"

        encoding = self.tokenizer.encode_plus(
            full_text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].flatten()  # 인코딩된 input_ids
        attention_mask = encoding['attention_mask'].flatten()  # 인코딩된 attention_mask

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': input_ids  # 레이블도 input_ids와 동일하게 설정
        }

# 데이터셋 분할
df = df.sample(frac=1, random_state=42).reset_index(drop=True)   # 데이터셋을 무작위로 섞기

# 훈련 데이터와 검증 데이터로 분리
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = EPLDataset(train_df, tokenizer)   # 학습 데이터셋 생성
val_dataset = EPLDataset(val_df, tokenizer)       # 검증 데이터셋 생성

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # 학습 데이터 로더 생성
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)     # 검증 데이터 로더 생성

# **3. 학습 및 평가 함수 설정**

In [9]:
# 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 또는 CPU 설정
model.to(device)  # 모델을 device로 이동

optimizer = AdamW(model.parameters(), lr=5e-5)  # 옵티마이저 설정
num_epochs = 100  # 초기 epoch 수를 크게 설정
num_training_steps = num_epochs * len(train_loader)  # 전체 학습 스텝 수
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)  # 스케줄러 설정

# 학습 함수
def train(model, dataloader, optimizer, scheduler, device):
    model.train()  # 모델을 학습 모드로 설정
    total_loss = 0  # 총 손실 초기화
    all_preds = []  # 모든 예측값 저장 리스트
    all_labels = []  # 모든 실제값 저장 리스트
    for batch in tqdm(dataloader):  # 배치마다 반복
        input_ids = batch['input_ids'].to(device)  # 배치의 input_ids를 device로 이동
        attention_mask = batch['attention_mask'].to(device)  # 배치의 attention_mask를 device로 이동
        labels = batch['labels'].to(device)  # 배치의 labels를 device로 이동

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)  # 모델의 출력값
        loss = outputs.loss  # 손실값

        total_loss += loss.item()  # 총 손실값에 더하기

        loss.backward()  # 손실값으로 역전파
        optimizer.step()  # 옵티마이저 스텝
        scheduler.step()  # 스케줄러 스텝
        optimizer.zero_grad()  # 옵티마이저 그래디언트 초기화

        # 예측 및 실제 레이블 저장 (평가를 위해)
        preds = torch.argmax(outputs.logits, dim=-1)
        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(labels.cpu().numpy().flatten())

    # 성능 지표 계산
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    return total_loss / len(dataloader), acc, f1  # 평균 손실, 정확도, F1 점수 반환

# 평가 함수
def evaluate(model, dataloader, device):
    model.eval()  # 모델을 평가 모드로 설정
    total_loss = 0  # 총 손실 초기화
    all_preds = []  # 모든 예측값 저장 리스트
    all_labels = []  # 모든 실제값 저장 리스트
    with torch.no_grad():  # 기울기 계산 비활성화
        for batch in dataloader:  # 배치마다 반복
            input_ids = batch['input_ids'].to(device)  # 배치의 input_ids를 device로 이동
            attention_mask = batch['attention_mask'].to(device)  # 배치의 attention_mask를 device로 이동
            labels = batch['labels'].to(device)  # 배치의 labels를 device로 이동

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)  # 모델의 출력값
            loss = outputs.loss  # 손실값

            total_loss += loss.item()  # 총 손실값에 더하기

            # 예측 및 실제 레이블 저장 (평가를 위해)
            preds = torch.argmax(outputs.logits, dim=-1)
            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(labels.cpu().numpy().flatten())

    # 성능 지표 계산
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    return total_loss / len(dataloader), acc, f1  # 평균 손실, 정확도, F1 점수 반환

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# **4. Early Stopping을 이용한 모델 학습**

In [10]:
import os
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience  # 조기 중단을 결정하기까지 기다리는 기간
        self.min_delta = min_delta  # 최소 개선량
        self.counter = 0  # 카운터 초기화
        self.best_loss = None  # 최고의 손실값 초기화
        self.early_stop = False  # 조기 중단 플래그 초기화

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss  # 초기 손실값 설정
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1  # 개선되지 않으면 카운터 증가
            if self.counter >= self.patience:
                self.early_stop = True  # 인내심 초과 시 조기 중단 플래그 설정
        else:
            self.best_loss = val_loss  # 새로운 최고의 손실값 업데이트
            self.counter = 0  # 카운터 초기화

# 저장할 디렉토리 생성 (존재하지 않는 경우)
save_dir = '/content/drive/MyDrive/fine_tuned_kogpt2'
os.makedirs(save_dir, exist_ok=True)

# 모델 학습
early_stopping = EarlyStopping(patience=5, min_delta=0.01)  # EarlyStopping 객체 생성, 인내심을 5로 설정하고 개선 최소값을 0.01로 설정

num_epochs = 100  # 총 epoch 수
for epoch in range(num_epochs):  # 총 epoch 수만큼 반복
    print(f"Epoch {epoch + 1}/{num_epochs}")  # 현재 epoch 번호 출력
    train_loss, train_acc, train_f1 = train(model, train_loader, optimizer, scheduler, device)  # 학습 데이터셋에 대해 모델 학습 및 성능 평가
    val_loss, val_acc, val_f1 = evaluate(model, val_loader, device)  # 검증 데이터셋에 대해 모델 평가

    # 학습 결과 출력
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}, F1 Score: {train_f1:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}")

    early_stopping(val_loss)  # EarlyStopping 객체에 현재 검증 손실값 전달
    if early_stopping.early_stop:  # 조기 중단 플래그가 설정되었는지 확인
        print("Early stopping")  # 조기 중단이 트리거되었음을 출력
        break  # 학습 중단

    # 5 epoch마다 모델과 토크나이저 저장
    if (epoch + 1) % 5 == 0:
        model_save_path = os.path.join(save_dir, f"model_epoch_{epoch + 1}")
        tokenizer_save_path = os.path.join(save_dir, f"tokenizer_epoch_{epoch + 1}")
        model.save_pretrained(model_save_path)
        tokenizer.save_pretrained(tokenizer_save_path)
        print(f"Model and tokenizer saved at epoch {epoch + 1}")

# 최종 모델과 토크나이저 저장
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Final model and tokenizer saved.")


Epoch 1/100


100%|██████████| 431/431 [04:34<00:00,  1.57it/s]


Train Loss: 0.3369, Accuracy: 0.8690, F1 Score: 0.8687
Validation Loss: 0.2511, Accuracy: 0.8462, F1 Score: 0.8451
Epoch 2/100


100%|██████████| 431/431 [04:33<00:00,  1.57it/s]


Train Loss: 0.1609, Accuracy: 0.8732, F1 Score: 0.8720
Validation Loss: 0.1805, Accuracy: 0.8463, F1 Score: 0.8453
Epoch 3/100


...

Train Loss: 0.0306, Accuracy: 0.8731, F1 Score: 0.8722
Validation Loss: 0.0977, Accuracy: 0.8462, F1 Score: 0.8452
Epoch 12/100


100%|██████████| 431/431 [04:34<00:00,  1.57it/s]


Train Loss: 0.0315, Accuracy: 0.8731, F1 Score: 0.8722
Validation Loss: 0.1000, Accuracy: 0.8462, F1 Score: 0.8453
Epoch 13/100


100%|██████████| 431/431 [04:33<00:00,  1.57it/s]


Train Loss: 0.0290, Accuracy: 0.8731, F1 Score: 0.8722
Validation Loss: 0.1053, Accuracy: 0.8462, F1 Score: 0.8452
Early stopping
Final model and tokenizer saved.


In [ ]:
torch.save(model.state_dict(), os.path.join(save_dir, f'model_epoch_{epoch + 1}.pth'))

In [ ]:
# 모델 저장
model.save_pretrained("/content/drive/MyDrive/fine_tuned")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned")

('/content/drive/MyDrive/fine_tuned/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned/tokenizer.json')

# **5. 질문-답변 테스트**

In [1]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# 모델 및 토크나이저 로드
model_path = "./fine_tuned_kogpt2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# 모델을 평가 모드로 전환
model.eval()

# 테스트를 위해 입력을 받는 함수 정의
def generate_response(question, model, tokenizer, max_length=50):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # 입력 텍스트 전처리
    input_text = f"질문: {question} 답변:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

    # 답변 생성
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=1,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            early_stopping=True
        )

    # 출력 텍스트 디코딩
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # "답변:" 이후의 텍스트만 추출
    response = response.split("답변:")[1].strip()

    return response

# 질문과 답변 반복 루프
while True:
    question = input("질문을 입력하세요 (종료하려면 '종료'를 입력하세요): ")
    if question.lower() == '종료':
        break

    response = generate_response(question, model, tokenizer)
    print(f"질문: {question}")
    print(f"답변: {response}")


/Users/muru/miniforge3/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


질문: 
답변: 아스톤빌라의 41번 선수는 제이콥 램지(Jacob Ramsey)입니다
질문: 첼시 감독
답변: 첼시의 감독은 엔초 마레스카 (Enzo Maresca)입니다 그는 탈리아의 축구 선수 출신으로 현재 첼시 FC의 감독을 맡고 있습니다
질문: 첼시 주장
답변: 첼시의 주장은 리스 제임스 (Reece James) 포지션 미드필더수비수 활약 기간 19531958 특징 맨유의 왼쪽 풀백으로 뛰었던 선수입니다
질문: 토트넘 홈구장
답변: 토트넘의 홈구장은 토트넘 홋스퍼 스타디움 (Tottenham Hotspur Stadium)입니다
